In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.mlab as mlab
import plotly as py
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns 
import pandas as pd
import statsmodels.api as sm
import numpy as np
import statistics as stats

In [ ]:
from sklearn import metrics
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
%matplotlib inline

In [ ]:
df_0620 = pd.read_csv('0620.csv', header=0)
df_0520 = pd.read_csv('0520.csv', header=0)
df_0420 = pd.read_csv('0420.csv', header=0)
df_0320 = pd.read_csv('0320.csv', header=0)
df_0220 = pd.read_csv('0220.csv', header=0)
df_0120 = pd.read_csv('0120.csv', header=0)
df_1219 = pd.read_csv('1219.csv', header=0)
df_1119 = pd.read_csv('1119.csv', header=0)
df_1019 = pd.read_csv('1019.csv', header=0)
df_0919 = pd.read_csv('0919.csv', header=0)
df_0819 = pd.read_csv('0819.csv', header=0)
df_0719 = pd.read_csv('0719.csv', header=0)

df_master = pd.concat([df_0719, df_0819, df_0919, df_1019, df_1119, df_1219, df_0120, df_0220, df_0320, df_0420, df_0520, df_0620])

#(df_master.head())

# Citi Bike, Jersey City Data

## Motivation 

We have started this analysis looking for ridership patterns that would allow us understand the reliability and importance of Citi Bikes as a mode of transportation in Jersey City. Originally, we were looking to work directly with New York City data, however, it is a significantly larger dataset as users make about 1.5M trips in NYC every month compared to approximately 31K trips in Jersey City. 

Our dataset is comprised of 12 months of data from July 2019 to June 2020. The data is publicly available and published directly from Citi Bike at https://www.citibikenyc.com/system-data. 

Each row is a trip and we have different types of data such as:
><b>-Trip duration</b> (in seconds)<br>
<b>-Trip start and end time</b> (datestamp)<br>
<b>-Start and end station name/id</b><br>
<b>-Coordinates of starting and ending stations</b><br>
<b>-Bikeid</b> which is the individual registry of the bike on Citi Bike systems<br>
<b>-Usertype:</b> Subscriber or Customer. Annual subscriptions cost \\$179 and customers get unlimited 45 rides. Customers can either pay \\$3 for a single trip, or \\$12 per day of unlimited 30 minute rides in 24 hours.<br>
<b>-Demographics</b> such as gender (0=Unknwown, 1=Male, 2=Female), and birth year<br>

We have added a few columns for visualization purposes such as duration in minutes, month of the year, day of the week, and hour of the day.

In [ ]:
df = df_master

df_master['starttime'] = pd.to_datetime(df_master['starttime'])

month = df_master['starttime'].dt.month_name()
df_master['month'] = month
#print(month)

weekday = df['starttime'].dt.day_name()
df_master['weekday'] = weekday
#print(weekday)

#df['starttime'] = pd.to_datetime(df['starttime'])
hour = df['starttime'].dt.hour
df_master['hour'] = hour
#print(hour)

#print(df_master.head())
#df_master.columns
df_master.isna().mean()

# How clean is our data?

The first part of our analysis was to check for missing values. The .isna() function was an efficient way of knowing if we were missing values, as it returns the percentage of N/A values in a column. As we can see, we are not missing any data in our columns.

In [ ]:
duration_minutes = df_master['tripduration']/60
df_master['duration_minutes'] = duration_minutes #add trip duration column in minutes

duration_hours = df_master['duration_minutes']/60
df_master['duration_hours'] = duration_hours #add trip duration column in hours

duration_days = df_master['duration_hours']/24 #add trip duration column in days
df_master['duration_days'] = duration_days

print(df_master.tripduration.describe())
print('===================================')
print(df_master.duration_minutes.describe())
print('===================================')
print(df_master.duration_hours.describe())
print('===================================')
print(df_master.duration_days.describe())
print('===================================')
print(df_master['birth year'].describe())
print('Dataframe shape is', df_master.shape)

# Exploratory Data Analysis (EDA)

We started our EDA by doing descriptive statistics in order to understand some of the most important information about our data. 

We see that the 'tripduration' column is not easy to visualize as it is measured in seconds. Therefore, we decided to convert it to minutes. Immediately we realized that the MAX value is extremely high as it converts to 29 days! Although we have no way of knowing from this data, we can easily assume this datapoint is incorrect and should be treated as an outlier that should be removed. For the sake of analysis, we will proceed without removing outliers as the datapoints may be attached to other important information. We also can see that our mean, and standard deviation are being influenced by the outliers, therefore we will be paying less attention to those values for now. 

We are also aware that birth year contains several outliers as the MIN value is 1887, we will also be keeping these outliers for visualization purposes and as the rest of the data may be valid. 

In [ ]:
birthyear = df_master['birth year']
Q1 = birthyear.quantile(.25)
Q3 = birthyear.quantile(.75)
IQR = Q3 - Q1
(IQR)

print('Gender: 0 = Unknown, 1 = Male, 2 = Female')
#print('Q3 value is', Q3)
#print('Q1 value is', Q1)
#print('The IQR is', IQR)
fig = px.box(df_master, x='gender', y='birth year', color='usertype', title='Boxplots by gender, birth year, usertype')
fig.show()

# Visualizing Outliers

Based on these boxplots, we see that we have a large number of outliers, this may be due to people choosing not to disclose their real age. We also see a third gender for people who chose not to disclose their information or may simply not identify as male or female.

As mentioned above, we won't be removing these outliers as the rest of the information may be valid.

In [ ]:
fig = px.histogram(df_master, x='birth year', color='usertype', title='Histogram by birth year and user type', opacity=0.75)
fig.show()

# Visualizing Data Distribution

On the histogram above, we see that the data distribution is skewed to the left. This is mostly driven by the large amount of erroneous birth years. We also notice the spike of rides where the user claims their birth year is '1969'. If we were to ignore the birth years prior to 1970, we could see a more bell-shaped distribution and get a better idea of who the riders actually are. 

In [ ]:
#need to fix legend

dict={"0": "Unknown", "1": "Male", "2": "Female"}
print('Gender: 0 = Unknown, 1 = Male, 2 = Female')
fig = px.histogram(df_master, x='usertype', color="gender", title="Histogram by usertype and gender")
fig.show()

# What type of users does Citi Bike have?

As mentioned earlier, Citi Bike has two types of users, Subscribers and Customers. 

Citi Bike's users in Jersey City are mostly Subscribers. Please note this doesn't mean Citi Bike has over 300K subscribers, it only means subscribers have taken more than 300K <i>rides</i>, which represents about 82% of total ridership during July 2019 and June 2020.

On the other hand, we see that customers are a small portion of the rides. It would be interesting to see the difference in revenue from Subscribers compared to Customers. Unfortunately that data is not provided by Citi Bike (or at least not that I am aware of) but it would be interesting to pursue as next steps if it is, and analyze the revenue drivers of Citi Bike. 

To summarize in a quick insight from this chart, <b>82% of rides started in Jersey City are taken by Citi Bike's Subscribers.</b>

In [ ]:
fig = px.histogram(df_master, x='month', title='Rides by month', color='usertype')
fig.show()

# Which months are the busiest for Citi Bike

This information could be helpful for Citi Bike managers trying to understand when to expect an influx of rides, and therefore, expect a higher need for additional staff members in order to keep Citi Bike running.

As one may expect, most rides are taken during the nice weather months of July through October, while ridership decreased by 38% in the months of November through March. 

April had a very small number of riders in 2020 which can potentially correlate with the strict lockdowns due to COVID-19. It would be interesting to see as next steps how April 2020 was different for riders than the month of April in previous years. 

May and June had an interesting approach as customers increased ridership compared to previous months. Conventional wisdom may tell us that tourists drive the number of customer rides in the summer months. However, tourism in NYC and Jersey City should have decreased in those months in 2020. Perhaps, as locals were less likely to use other vehicles such as cars or subways, they turned to Citi Bike as it can be a fairly safe mode of transportation as long as you take the necessary precautions. 

A separate analysis that compares the months of May through July for 2019 and 2020, shows that <b>ridership by customers increased by 1.82 times in 2020 while subscribers ridership decreased by about 45%</b> <i>(Nathan: Do you know how I can reference this?, the analysis was made by me in a different Jupyter notebook.)</i>

As next steps, it would be interesting to see how this shifts and what the future of mobility in NYC and metropolitan area would look like. 

In [ ]:
fig = px.histogram(df_master, x='weekday', title='Rides by weekday', color='usertype')
fig.show()

# What days are Citi Bikes used the most?

Subscribers used the bikes mostly between Monday and Friday, and less over the weekends. 
Customers used the bikes mostly on the weekends. 

Conventional wisdom tells us that <b>Subscribers are more likely to use Citi Bike to commute and customers may use it more for recreational purposes.</b>

In [ ]:
#fig = px.histogram(df_master, x='hour', title='Rides by hour', color='usertype')
#fig.show()

# Drill down to hours

Drilling down, the hours Citi Bikes are used the most depend on the type of user, we found a pattern that may support our previous point. <b>Subscribers are riding Citi Bikes mostly at 8 AM and 6 PM</b>, while <b>Customers use it more often between the afternoon and early evening.</b> 

In [ ]:
fig = px.pie(df_master, names='start station name', title='Bike Stations by % of total trips started')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

In [ ]:
fig = px.pie(df_master, names='end station name', title='Bike Stations by % of total trips ended')
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

# What stations are used the most?

The pie charts above show us the percentage of rides started from and ended at specific bike stations. We see that <b>Grove St. PATH and Hamilton Park were the two stations where most rides were started from,</b> with a difference of about <b>15K more rides being taken from Grove St. PATH compared to Hamilton Park</b>

On the second pie chart we see that <b>12.4% rides (46k rides) end at Grove St. PATH.</b>

This information could be valuable to operations managers whose job is to make sure the appropriate balance of bikes is achieved, otherwise a consumer could be turned away and use a different mode of transportation which would mean less revenue earned by Citi Bike. As next steps, we could look at the hourly patterns of specific stations to see when to expect higher demand and proactively rebalance the number of bikes.

In [ ]:
start_station = df_master['start station name']
#print(start_station)
end_station = df_master['end station name']
#print(end_station)

df_master['most frequent trips'] = start_station + ' to '+ end_station
#print(df_master.head())
trips1 = df_master.groupby(['start station name', 'start station id', 'end station name', 'end station id', 'most frequent trips']).size().reset_index(name = 'number of trips')

trips_desc1 = trips1.sort_values('number of trips', ascending=False)
trips_desc = (trips_desc1.head(10).reset_index())
#trips_desc

In [ ]:
#x=trips_desc['number of trips']
#y=trips_desc['most frequent trips']

#fig = go.Figure(go.Bar(x=x, y=y, orientation='h'))

#fig.update_layout(title='Top 10 most frequent trips',
#                  xaxis_title='number of trips',
#                  yaxis_title='trips',
#                  yaxis={'categoryorder':'total ascending'})
#fig.show()

# Where are people going?

No way of knowing for sure, however, we can tell you the routes that were taken the most. Turns out Hamilton Park to Grove St. PATH, is about .7 miles and there is a PATH train station nearby. For those of us that don't live in NYC and metropolitan area, PATH is the Port Authority Trans-Hudson train system that connects northern New Jersey and Lower and Midtown Manhattan. Interestingly enough, <b>6 of the 10 most common routes end at Grove St. PATH.</b>

In [ ]:
Grove_st_PATH = df_master['start station name'] == 'Grove St PATH' #creates filter by station name
#print(Grove_st_PATH.head())

Grove_st_PATH = df_master[Grove_st_PATH] #applies filter to the df 
#(Grove_st_PATH)

In [ ]:
fig = px.histogram(Grove_st_PATH, x='hour', title='Rides by hour from Groves St. PATH', color='usertype')
#fig.show()

In [ ]:
Grove_st_PATH_end = df_master['end station name'] == 'Grove St PATH' #creates filter by station name
#print(Grove_st_PATH_end.head())

Grove_st_PATH_end = df_master[Grove_st_PATH_end] #applies filter to the df 
#(Grove_st_PATH_end)

In [ ]:
fig = px.histogram(Grove_st_PATH_end, x='hour', title='Rides by hour ending at Groves St. PATH', color='usertype')
#fig.show()

# Groves St. PATH 

Based on the data, we see that most rides <i>end</i> at Groves St. PATH between 7 and 8 AM. Additionally, more rides <i>start</i> from this station in the early evening hours, more specifically between 5 and 7 PM. 

In [ ]:
Grove_st_PATH = df_master['start station name'] == 'Grove St PATH' #creates filter by station name
#print(Grove_st_PATH.head())

Grove_st_PATH = df_master[Grove_st_PATH] #applies filter to the df 
(Grove_st_PATH.shape)
#(Grove_st_PATH)

Month = Grove_st_PATH['month'] == 'July' #creates filter by month, in this case we filtered out only values from July, 2019
#print(Grove_st_PATH.head())

Month = Grove_st_PATH[Month] #applies month filter to the df 
(Month.shape)
(Month)

dayofweek = Month['weekday'] == 'Wednesday' #creates filter by random weekday, in this case we filtered to Wednesday
#print(Grove_st_PATH.head())

dayofweek = Month[dayofweek] #applies weekday filter to the df 

df719 = dayofweek

#(df719)

In [ ]:
#Group by our data. Below we have the total number of trips taken on Wednesdays of July 2019 grouped by hour of the day.

df719 = df719.groupby(['start station name', 'hour']).size().reset_index(name = 'number of trips')

df719 = df719.sort_values('number of trips', ascending=False)
df719 = (df719.reset_index())
df719

In [ ]:
df720 = dayofweek.groupby(['start station name', 'hour']).size().reset_index(name = 'number of trips')

df720 = station1.sort_values('number of trips', ascending=False)
df720 = (station1_desc.reset_index())
df720

# Linear vs. Non-Linear Regressions

The problem we are interested in solving is having the right amount of bikes, at the right time, at the right place. For this we decided to apply regression models such as Decistion Tree and Random Forest, as well as Ordinary Least Squares (OLS) to see which regression model would help us predict the needs of Citi Bikes users more accurately. 

The scatter plot below shows filtered trips taken from a specific station (Grove St. PATH) on every Wednesday of July 2019. Our X variable is time (hour of the day) and we are trying to predict Y, which is the number of bikes needed for the month of July 2020.

# Forecast vs. Actual number of rides taken in July 2020 (OLS)

Below we see an R-squared of 0.191 which means our data can explain only 19% the increase in Y based on X, that is only 19% of rides taken are due to an increase in the hour of the day. We also see that the linear model predicts that Citi Bike will see an increase in ridership by every hour that passes which we know not to be entirely accurate as less people ride during the late hours of the night. 

The first scatter plot shows the filtered data for July 2019 along with the forecast for July 2020, and the second scatter plot shows the actual number of rides taken during July 2020.

Unfortunately, due to COVID-19, we can't really expect the forecast to be entirely accurate as trends have shifted from 2019 to 2020 but we tried to control for as many variables as possible. 

In [ ]:
X = df719['hour']
X = sm.add_constant(X)
y = df719['number of trips']
OLSmodel = sm.OLS(y, X)
OLSmodelResult = OLSmodel.fit()
OLSmodelResult.summary()

In [ ]:
import plotly.express as px

fig = px.scatter(df719, x='hour', y="number of trips", trendline="ols", title='July 2019 data and forecast')
fig.show()

fig = px.scatter(df720, x='hour', y="number of trips", trendline="auto", title='July 2020 actual data')
fig.show()

# Decision Tree and Random Forest

Below we make an attempt at understanding what a Decision Tree and Random Forest is, and how it works. Even though the numbers seem pretty strong, we are not completely sure on what they help us predict. The example we followed can be found at https://www.pluralsight.com/guides/non-linear-regression-trees-scikit-learn and it's their code entirely. 

In [ ]:
target_column = ['number of trips'] 
predictors = ['hour']
df719[predictors] = df719[predictors]/df719[predictors].max()
station1.describe()

In [ ]:
X = df719[predictors].values
y = df719[target_column].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape); print(X_test.shape)

In [ ]:
dtree = DecisionTreeRegressor(max_depth=8, min_samples_leaf=0.13, random_state=3)

dtree.fit(X_train, y_train)

In [ ]:
# Code lines 1 to 3
pred_train_tree= dtree.predict(X_train)
print('RMSE for Train Data', (np.sqrt(mean_squared_error(y_train,pred_train_tree))))
print('R Squared for Train Data', (r2_score(y_train, pred_train_tree)))

# Code lines 4 to 6
pred_test_tree= dtree.predict(X_test)
print('RMSE for Test Data', (np.sqrt(mean_squared_error(y_test,pred_test_tree))))
print('R Squared for Test Data', (r2_score(y_test, pred_test_tree)))

In [ ]:
# Code Lines 1 to 4: Fit the regression tree 'dtree1' and 'dtree2' 
dtree1 = DecisionTreeRegressor(max_depth=2)
dtree2 = DecisionTreeRegressor(max_depth=5)
dtree1.fit(X_train, y_train)
dtree2.fit(X_train, y_train)

# Code Lines 5 to 6: Predict on training data
tr1 = dtree1.predict(X_train)
tr2 = dtree2.predict(X_train) 

#Code Lines 7 to 8: Predict on testing data
y1 = dtree1.predict(X_test)
y2 = dtree2.predict(X_test) 

In [ ]:
# Print RMSE and R-squared value for regression tree 'dtree1' on training data
print(np.sqrt(mean_squared_error(y_train,tr1))) 
print(r2_score(y_train, tr1))

# Print RMSE and R-squared value for regression tree 'dtree1' on testing data
print(np.sqrt(mean_squared_error(y_test,y1))) 
print(r2_score(y_test, y1)) 

In [ ]:
# Print RMSE and R-squared value for regression tree 'dtree2' on training data
print(np.sqrt(mean_squared_error(y_train,tr2))) 
print(r2_score(y_train, tr2))

# Print RMSE and R-squared value for regression tree 'dtree2' on testing data
print(np.sqrt(mean_squared_error(y_test,y2))) 
print(r2_score(y_test, y2)) 

In [ ]:
#RF model
model_rf = RandomForestRegressor(n_estimators=500, oob_score=True, random_state=100)
model_rf.fit(X_train, y_train) 
pred_train_rf= model_rf.predict(X_train)
print(np.sqrt(mean_squared_error(y_train,pred_train_rf)))
print(r2_score(y_train, pred_train_rf))

pred_test_rf = model_rf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,pred_test_rf)))
print(r2_score(y_test, pred_test_rf))

# Forecast vs. Actual number of rides taken in July 2020 (Non-Linear Regression)

We found that the number of rides decreased in July 2020 to 36% of the total number of rides in July 2019. However, the non-linear regression and trendline were surprisingly accurate, particularly in the early hours of the evening. 

In [ ]:
fig = px.scatter(df719, x='hour', y="number of trips", trendline="lowess", title='July 2019 data with forecast')
fig.show()

fig = px.scatter(df720, x='hour', y="number of trips", trendline="auto", title='Actual number of rides taken, July 2020')
fig.show()

# Conclusion and Next Steps

With this analysis, we were able to understand the different patterns of Citi Bike ridership in Jersey City. We realized that the users that ride the most are male subscribers and that the median age for the entire dataset population is 1984. This data is considering an extensive amount of rides where the age is erroneous as riders are skeptical of sharing their actual age to Citi Bike, however, their trip information is valuable for the rest of the project. 

With this analysis, we also were able to draw preliminary conclusions due to ridership patterns in specific months, days, and hours, and determined that it is likely that most rides are as part of a daily commute during the weekdays and perhaps more recreational over the weekends. 

We also noted which stations are used the most to start and end a trip. We believe this information to be important as it can have a direct impact on bike availability and increased customer satisfaction. We also believe that failing to pay attention to these details could negatively impact revenue and profitability of Citi Bike. 

From an inferential statistics standpoint, we find that the nature of our data is not linear and the best way to predict would be to use non-linear regressions which unfortunately are out of scope for us at this moment. We see that the non-linear trendline is much more accurate during the early hours of the day/evening but the late hours of the night aren't quite at the same level of accuracy. A preliminary thought is this may be due to COVID-19 and the change in behavior we mentioned earlier, such as the decreased need to be out late at night at bars or coming back from the office or school. 

As noted throughtout the analysis, the next steps we would be interested in looking at to further develop this project would be the following: 

>-Analyze the revenue drivers of Citi Bike to further understand what kind of financial implications different marketing decisions would create<br>
<br>
-How April 2020 was different for Citi Bike compared to the same month in previous years, and the impact of COVID-19 in ridership in subsequent months<br>
<br>
-How does ridership and the future of mobility in NYC and metropolitan area shifts post-pandemic. Would people be more interested in biking to work if infrastructure was improved as it may be safer than a subway train during a public health contingency?, what would happen if people work from home more often and need Citi Bike less?<br></b>

In [ ]:
#Adding month of July 2020 

df720 = pd.read_csv('0720.csv')


In [ ]:
df720['starttime'] = pd.to_datetime(df720['starttime'])

weekday = df720['starttime'].dt.day_name()
df720['weekday'] = weekday

hour = df720['starttime'].dt.hour
df720['hour'] = hour


In [ ]:
Grove_st_PATH20 = df720['start station name'] == 'Grove St PATH' #creates filter by station name
#print(Grove_st_PATH.head())

Grove_st_PATH20 = df720[Grove_st_PATH20] #applies filter to the df 
(Grove_st_PATH20)

dayofweek20 = Grove_st_PATH20['weekday'] == 'Wednesday' #creates filter by random weekday, in this case we filtered to Wednesday

dayofweek20 = Grove_st_PATH20[dayofweek20] #applies weekday filter to the df 
(dayofweek20)

df720 = dayofweek20


In [ ]:
#X = df720['hour']
#X = sm.add_constant(X)
#y = df720['number of trips']
#OLSmodel = sm.OLS(y, X)
#OLSmodelResult = OLSmodel.fit()
#OLSmodelResult.summary()